In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="gemma3:1b")

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調及繁體中文回覆。禁止提供結語"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [ ]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [ ]:
import time


def add_Conclusion_text(content:str)->dict:
    now_time = time.strftime("%Y/%m/%d %H:%M")
    result = f"""{content}


ABC團隊 敬上
回覆時間：{now_time}"""
    return {"formatted_reply":result}

In [ ]:
from langchain.schema.runnable import RunnableLambda
chain = (prompt_template 
         | model 
         | StrOutputParser()
         | RunnableLambda(add_Conclusion_text)
         | improve_prompt
         | model
         | StrOutputParser()
)

In [ ]:
customer_email = """您好，

我是昨天在貴公司網站購買商品的客戶。收到商品後發現有損壞的情況，想請問如何辦理換貨或是退款？我可以提供損壞商品的照片以及訂單編號，請協助處理，謝謝！

請盡快回覆，感謝您！

祝順心
"""

In [ ]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)